In [1]:
import numpy as np
import pandas as pd

# predict: diabetes_012

# Задание 1

Какова вероятность отправиться на прогулку если идёт дождь, при наличии следующих наблюдений?

## Формула Байеса

Формула Байеса позволяет «переставить причину и следствие»: по известному факту события вычислить вероятность того, что оно было вызвано данной причиной. События, отражающие действие «причин», в данном случае называют гипотезами, так как они — предполагаемые события, повлекшие данное.

$P(A|B) =\frac{P(A) \cdot P(B | A)}{P(B)}$ (1)

* $P(A)$  -- априорная вероятность гипотезы A;
* $P(A|B)$ --  вероятность гипотезы A при наступлении события B (апостериорная вероятность);
* $P(B|A)$ --  вероятность наступления события B при истинности гипотезы A;
* $P(B)$ -- полная вероятность наступления события B.

In [2]:
data = [
        ('солнечно', True),
        ('снег', False),
        ('облачно', False),
        ('дождь', False),
        ('солнечно', True),
        ('снег', False),
        ('облачно', True),
        ('снег', False),
        ('солнечно', False),
        ('облачно', True),
        ('снег', True),
        ('солнечно', True),
        ('дождь', False),
        ('дождь', True),
        ('облачно', True),
]

* A -- мужик отправился на прогулку
* B -- идёт дождь

In [9]:
P_A = sum(i[1] for i in data) / len(data)
P_B = sum(i[0] == 'дождь' for i in data) / len(data)
P_BA = sum(i[0] == 'дождь' and i[1] for i in data) / sum(i[1] for i in data)

In [10]:
result = P_A * P_BA / P_B
print(result)


0.3333333333333333


### Задание 2

- загрузите датасет для классификации, проведите предобработку, выделите целевой признак и предикторы, разбейте данные на обучающую и тестовую выборку;
- Масштабируйте числовые признаки
- В случае дисбаланса проведите балансировку
- решите задачу классификации на ваших данных с использованием рассмотренных моделей sklearn: knn, NB, Logistic Regression, SVM. Не забудьте подобрать гиперпараметры.
- вычислите значения метрик Accuracy, Precision, Recall, F1, ROC AUC score


## Загрузка датасета

In [206]:
data = pd.read_csv('../datasets/raw/diabetes.csv')
data = data.sample(frac=1)
data

,Diabetes_012,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
103380,0.0,0.0,0.0,1.0,22.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,3.0,0.0,0.0,0.0,0.0,9.0,5.0,8.0
76776,0.0,1.0,0.0,1.0,29.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,2.0,0.0,0.0,0.0,1.0,6.0,5.0,8.0
42585,2.0,1.0,0.0,1.0,28.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,9.0,6.0,6.0
248205,0.0,0.0,1.0,1.0,25.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,0.0,0.0,0.0,0.0,11.0,5.0,8.0
88652,0.0,0.0,0.0,1.0,29.0,1.0,0.0,0.0,1.0,1.0,...,1.0,0.0,3.0,0.0,8.0,0.0,0.0,6.0,3.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3351,0.0,0.0,0.0,1.0,46.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,25.0,0.0,0.0,0.0,4.0,6.0,6.0
211119,0.0,1.0,1.0,1.0,30.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,4.0,0.0,0.0,0.0,1.0,9.0,6.0,5.0
628,0.0,1.0,0.0,1.0,32.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,3.0,0.0,1.0,0.0,1.0,5.0,6.0,8.0
199882,0.0,0.0,0.0,1.0,23.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,0.0,10.0,0.0,0.0,7.0,6.0,6.0


In [207]:
data.isnull().values.any()

False

In [208]:
# data = data[data['Diabetes_012'] != 1]

## Балансировка

In [209]:
from sklearn.utils import resample

In [210]:
data_0 = data[data['Diabetes_012'] == 0]
data_1 = data[data['Diabetes_012'] == 1]
data_2 = data[data['Diabetes_012'] == 2]

print(data_0.shape, data_2.shape)

(213703, 22) (35346, 22)


In [211]:
data_0 = resample(data_0, replace=False, n_samples=7500)
data_1 = resample(data_1, replace=True, n_samples=7500)
data_2 = resample(data_2, replace=True, n_samples=7500)

print(data_0.shape, data_2.shape)

(7500, 22) (7500, 22)


In [212]:
data = pd.concat([data_0, data_1, data_2])
data['Diabetes_012'].groupby(data['Diabetes_012']).count()


Diabetes_012
0.0    7500
1.0    7500
2.0    7500
Name: Diabetes_012, dtype: int64

In [213]:
from sklearn.utils import shuffle
data = shuffle(data)

In [214]:
y = data['Diabetes_012']
X = data.drop(columns=['Diabetes_012'])

y.shape

(22500,)

## Разбитие на выборки

In [12]:
test_share = 0.8


test_count = int(data.shape[0] * (1 - test_share))

AttributeError: 'list' object has no attribute 'shape'

In [216]:
X_test = X.iloc[:test_count:]
X_train = X.iloc[test_count:,:]

y_test = y.iloc[:test_count]
y_train = y.iloc[test_count:]


## Масштабирование признаков

In [217]:
from sklearn.preprocessing import StandardScaler

In [218]:
scaler = StandardScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
X_train

,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,0.851504,0.899323,0.149591,0.259287,1.035805,-0.253632,2.410564,0.657053,0.798527,0.529604,...,0.228064,-0.34604,0.102017,-0.472637,-0.199818,1.684686,-0.904488,0.810944,0.115424,-0.268172
1,0.851504,-1.111947,0.149591,1.654254,1.035805,-0.253632,-0.414841,-1.521947,-1.252306,0.529604,...,0.228064,-0.34604,0.102017,0.001977,-0.493956,1.684686,-0.904488,1.172080,0.115424,-0.268172
2,0.851504,-1.111947,0.149591,0.259287,-0.965432,-0.253632,-0.414841,-1.521947,0.798527,0.529604,...,0.228064,-0.34604,-0.812532,-0.472637,-0.592002,-0.593582,1.105597,0.449809,0.115424,0.189008
3,0.851504,0.899323,0.149591,0.119790,-0.965432,-0.253632,2.410564,0.657053,0.798527,-1.888202,...,0.228064,-0.34604,1.016566,-0.472637,-0.592002,1.684686,1.105597,1.172080,1.073792,1.103370
4,0.851504,-1.111947,-6.684895,1.235763,1.035805,-0.253632,-0.414841,-1.521947,-1.252306,0.529604,...,0.228064,-0.34604,1.016566,-0.472637,-0.592002,1.684686,1.105597,1.172080,-0.842945,1.103370
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17996,-1.174392,0.899323,0.149591,1.096267,1.035805,-0.253632,-0.414841,-1.521947,-1.252306,0.529604,...,0.228064,-0.34604,1.016566,3.086970,2.349380,-0.593582,-0.904488,-0.633597,0.115424,1.103370
17997,-1.174392,-1.111947,0.149591,-1.135680,-0.965432,-0.253632,-0.414841,0.657053,0.798527,0.529604,...,0.228064,-0.34604,-1.727080,-0.472637,-0.592002,-0.593582,1.105597,-2.800408,1.073792,0.646189
17998,0.851504,0.899323,0.149591,1.654254,-0.965432,-0.253632,-0.414841,0.657053,0.798527,-1.888202,...,0.228064,-0.34604,-0.812532,-0.472637,-0.592002,1.684686,-0.904488,0.449809,-0.842945,-0.725353
17999,0.851504,-1.111947,0.149591,-0.159203,-0.965432,-0.253632,-0.414841,0.657053,-1.252306,0.529604,...,0.228064,-0.34604,-0.812532,-0.472637,-0.592002,-0.593582,1.105597,0.088674,1.073792,0.646189


# sklearn kNN

In [322]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=6, metric='minkowski', p=2)

knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=6)

In [323]:
y_knn_pred = knn.predict(X_test)

In [324]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_knn_pred)

0.3731940431206935

In [245]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_knn_pred)

array([[1230,   38,  208],
       [1136,   20,  369],
       [1052,   17,  429]], dtype=int64)

In [246]:
from sklearn.metrics import classification_report


print(classification_report(y_test, y_knn_pred))

              precision    recall  f1-score   support

         0.0       0.36      0.83      0.50      1476
         1.0       0.27      0.01      0.03      1525
         2.0       0.43      0.29      0.34      1498

    accuracy                           0.37      4499
   macro avg       0.35      0.38      0.29      4499
weighted avg       0.35      0.37      0.29      4499



In [11]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score

roc_auc_score(y_test, knn.predict_proba(X_test), multi_class='ovo')

NameError: name 'y_test' is not defined

# sklearn NB

In [287]:
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()
gnb.fit(X_train, y_train)

GaussianNB()

In [288]:
y_gnb_pred = gnb.predict(X_test)

In [250]:
accuracy_score(y_test, y_gnb_pred)

0.33251833740831294

In [251]:
confusion_matrix(y_test, y_gnb_pred)

array([[   0,    1, 1475],
       [   0,    0, 1525],
       [   0,    2, 1496]], dtype=int64)

In [252]:
print(classification_report(y_test, y_gnb_pred))

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00      1476
         1.0       0.00      0.00      0.00      1525
         2.0       0.33      1.00      0.50      1498

    accuracy                           0.33      4499
   macro avg       0.11      0.33      0.17      4499
weighted avg       0.11      0.33      0.17      4499



C:\Users\Егор\Desktop\programming\ML\lab1\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Егор\Desktop\programming\ML\lab1\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Егор\Desktop\programming\ML\lab1\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

In [253]:
roc_auc_score(y_test, gnb.predict_proba(X_test), multi_class='ovo')

0.5502072076771823

# sklearn LR

In [192]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(solver='liblinear')
lr.fit(X_train, y_train)

LogisticRegression(solver='liblinear')

In [193]:
y_lr_pred = lr.predict(X_test)

In [194]:
accuracy_score(y_test, y_lr_pred)

0.32888814802467076

In [195]:
confusion_matrix(y_test, y_lr_pred)

array([[   0,    0, 2032],
       [   0,    0, 1994],
       [   0,    0, 1973]], dtype=int64)

In [196]:
print(classification_report(y_test, y_lr_pred))

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00      2032
         1.0       0.00      0.00      0.00      1994
         2.0       0.33      1.00      0.49      1973

    accuracy                           0.33      5999
   macro avg       0.11      0.33      0.16      5999
weighted avg       0.11      0.33      0.16      5999



C:\Users\Егор\Desktop\programming\ML\lab1\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Егор\Desktop\programming\ML\lab1\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Егор\Desktop\programming\ML\lab1\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

In [197]:
roc_auc_score(y_test, lr.predict_proba(X_test), multi_class='ovo')

0.5602544696680752

# Задание 3

## Accuracy

In [198]:
def accuracy(y_test, y_pred):
    if y_test.shape != y_pred.shape:
        return
    y_test = list(y_test)
    y_pred = list(y_pred)
    return sum([ y_test[i] == y_pred[i] for i in range(len(y_test))]) / len(y_test)

accuracy(y_test, y_knn_pred)

0.3802300383397233

## Precision

In [199]:
def precision(y_test, y_pred):
    if y_test.shape != y_pred.shape:
        return
    y_test = list(y_test)
    y_pred = list(y_pred)
    
    preds = {
        
    }
    
    result = {
        
    }
    
    for i in set(y_test):
        pred_right = sum([y_test[j] == y_pred[j] and y_test[j] == i for j in range(len(y_test))])
        pred_total = sum([y_pred[j] == i for j in range(len(y_test))])
        preds[i] = pred_right, pred_total
        
    for i in set(y_test):
        result[i] = round(preds[i][0] / preds[i][1] if preds[i][1] else 0, 2)
        
    return result

precision(y_test, y_gnb_pred)


{0.0: 0, 1.0: 0.0, 2.0: 0.33}

## Recall

In [200]:
def recall(y_test, y_pred):
    if y_test.shape != y_pred.shape:
        return
    y_test = list(y_test)
    y_pred = list(y_pred)
    
    preds = {
        
    }
    
    actual = {
        
    }
    
    result = {
        
    }
    
    for i in set(y_test):
        pred_right = sum([y_test[j] == y_pred[j] and y_test[j] == i for j in range(len(y_test))])
        pred_total = sum([y_pred[j] == i for j in range(len(y_test))])
        actual = sum([y_test[j] == i for j in range(len(y_test))])

        preds[i] = pred_right, pred_total, actual
    
    for i in set(y_test):
        result[i] = round(preds[i][0] / (preds[i][2]), 2)
        
    return result
    
recall(y_test, y_knn_pred)

{0.0: 0.76, 1.0: 0.2, 2.0: 0.17}

In [223]:
def F1(y_test, y_pred):
    result = {}
    for i in set(list(y_test)):
        if precision(y_test, y_pred)[i] + recall(y_test, y_pred)[i] == 0:
            result[i] = 0
            continue
        result[i] = round((2 * precision(y_test, y_pred)[i] * recall(y_test, y_pred)[i]) / (precision(y_test, y_pred)[i] + recall(y_test, y_pred)[i]), 2)
        
    return result
        
F1(y_test, y_knn_pred)

{0.0: 0.52, 1.0: 0.19, 2.0: 0.36}

# kNN

In [284]:
def dist(point, points):
    return np.sqrt(np.sum((point - points)**2, axis=1))

class KNearNeighbors():
    def __init__(self, k=5):
        self.k = k
        self.dist = dist
        
    def fit(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train
        
    def predict(self, X):
        
        neighbors = []
        
        for i in X.to_numpy():
            distances = self.dist(i, self.X_train)
            dists_sorted = [j[1] for j in sorted(zip(distances, self.y_train))]
            neighbors.append(dists_sorted[:self.k])
            
        result = []
        
        for n in neighbors:
            result.append(max([(i, n.count(i)) for i in n], key=lambda x: x[1])[0])
            
        return result
            
    def score(self, y_pred, y_test):
        accuracy = sum([y_pred[i] == list(y_test)[i] for i in range(len(y_pred))]) / len(y_test)
        return accuracy

In [285]:
knn1 = KNearNeighbors()
knn1.fit(X_train, y_train)

y_knn1_pred = knn1.predict(X_test)

In [286]:
score(y_knn1_pred, y_test)

0.3769726605912425

# NB

In [546]:
class Bayes():
#     def __init__(self):
#         pass
    
    def fit(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train
        
        self.features = list(X_train.columns)
        self.classes = set(y_train.unique())
        
        self.Py = {}
    
        for cl in self.classes:
            self.Py[cl] = sum(y_train.to_numpy()[i] == cl for i in range(len(y_train))) / len(y_train)
        
        
    def predict(self, X_test):
        
        print(list(y_test)[:25])
        
        result = []
        
        for el in X_test.to_numpy():
            probs = []
            for cl in self.classes:
                Pxy = self.Py[cl]
                
                for feature in self.features:
                    Pxy *= sum(y_train.to_numpy()[i] == float(cl) and (X_train.iloc[i][feature] - el[self.features.index(feature)])**2 < 1 for i in range(len(y_train)))
                Px = sum((X_train.iloc[i][feature] - el[self.features.index(feature)])**2 < 1 for i in range(len(X_train)))
                
#                 print([(y_train.to_numpy()[i] == float(cl), ((X_train.iloc[i][feature] - el[self.features.index(feature)]))**2 < 4) for i in range(100)])
                
#                 print([(X_train.iloc[i][feature] - el[self.features.index(feature)])**2  for i in range(100)])
                
                probs.append((cl, (Pxy / Px) if Px else 0))
                print((cl, (Pxy / Px) if Px else 0))
                
            result.append(max(probs, key=lambda x: x[1])[0])
            print(probs)
            
                
        return result
            
                
            
    
    def score(self):
        pass
    


In [547]:
NB = Bayes()

In [548]:
NB.fit(X_train, y_train)

In [549]:
NB.predict(X_test)

[2.0, 0.0, 0.0, 2.0, 2.0, 1.0, 1.0, 2.0, 1.0, 0.0, 1.0, 1.0, 1.0, 2.0, 2.0, 0.0, 2.0, 2.0, 1.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0]
(0.0, 0)
(1.0, 0)
(2.0, 0)
[(0.0, 0), (1.0, 0), (2.0, 0)]
(0.0, 0)
(1.0, 0)
(2.0, 0)
[(0.0, 0), (1.0, 0), (2.0, 0)]
(0.0, 0)
(1.0, 0)
(2.0, 0)
[(0.0, 0), (1.0, 0), (2.0, 0)]
(0.0, 0)
(1.0, 0)
(2.0, 0)
[(0.0, 0), (1.0, 0), (2.0, 0)]
(0.0, 0)
(1.0, 0)
(2.0, 0)
[(0.0, 0), (1.0, 0), (2.0, 0)]
(0.0, 0)
(1.0, 0)
(2.0, 0)
[(0.0, 0), (1.0, 0), (2.0, 0)]
(0.0, 0)
(1.0, 0)
(2.0, 0)
[(0.0, 0), (1.0, 0), (2.0, 0)]
(0.0, 0)
(1.0, 0)
(2.0, 0)
[(0.0, 0), (1.0, 0), (2.0, 0)]


KeyboardInterrupt: 